In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re

%matplotlib inline

In [2]:
df_train = pd.read_csv('input/train.csv')
# df = df[pd.notnull(df['tags'])]
df_train.head(10)

,itemid,title,Category,image_path
0,307504,nyx sex bomb pallete natural palette,0,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg
1,461203,etude house precious mineral any cushion pearl...,1,beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg
2,3592295,milani rose powder blush,2,beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg
3,4460167,etude house baby sweet sugar powder,3,beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg
4,5853995,bedak revlon color stay aqua mineral make up,3,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg
5,6208490,dr pure whitening cream,4,beauty_image/77e6b7e9d5544adbfda6809b2351c4fa.jpg
6,6959546,chanel powder blush malice,2,beauty_image/485c4b8435a1e94976a569f3f014ea8b.jpg
7,8984461,snail white cream original 100,4,beauty_image/c7336f1c2e590d4c2bee219a6a0351a7.jpg
8,9184082,sunprise all proof spf 50,4,beauty_image/970b32aa659689f371516e5ceb423e25.jpg
9,10345338,eyebrow powder nyx satuan rp 15.000 pc,3,beauty_image/49a47745248031775c98593f70f7201a.jpg


In [3]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, TimeDistributed, Embedding
from keras.preprocessing import text, sequence
from keras import utils

Using TensorFlow backend.


In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1']

In [5]:
train_posts, test_posts, train_tags, test_tags = train_test_split(df_train['title'], 
                                                                  df_train['Category'], 
                                                                  test_size=0.3, 
                                                                  random_state=40)

In [6]:
max_words = 2000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [7]:
tokenize.fit_on_texts(df_train['title']) # only fit on train
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [8]:
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [9]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [10]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (466630, 2000)
x_test shape: (199985, 2000)
y_train shape: (466630, 58)
y_test shape: (199985, 58)


In [11]:
batch_size = 512
epochs = 20

In [12]:
x_train[1,:]

array([0., 1., 1., ..., 0., 0., 0.])

In [13]:
train_posts[1]

'etude house precious mineral any cushion pearl aura puff'

In [14]:
# Build the model
model = Sequential()
model.add(LSTM(1000,input_shape=(1,max_words)))
model.add(Dropout(0.8))
model.add(Dense(512, activation='elu'))
model.add(Dropout(0.8))
model.add(Dense(num_classes, activation='elu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='elu'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
history = model.fit(x_train.reshape((x_train.shape[0],1,x_train.shape[1])), y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 419967 samples, validate on 46663 samples
Epoch 1/20
419967/419967 [==============================] - 44s 105us/step - loss: 1.4820 - acc: 0.6004 - val_loss: 1.0312 - val_acc: 0.6910
Epoch 2/20
419967/419967 [==============================] - 38s 90us/step - loss: 1.1680 - acc: 0.6642 - val_loss: 0.9926 - val_acc: 0.6966
Epoch 3/20
419967/419967 [==============================] - 38s 90us/step - loss: 1.1288 - acc: 0.6712 - val_loss: 0.9746 - val_acc: 0.6987
Epoch 4/20
419967/419967 [==============================] - 38s 90us/step - loss: 1.1039 - acc: 0.6758 - val_loss: 0.9638 - val_acc: 0.7012
Epoch 5/20
419967/419967 [==============================] - 38s 90us/step - loss: 1.0877 - acc: 0.6793 - val_loss: 0.9588 - val_acc: 0.7018
Epoch 6/20
419967/419967 [==============================] - 38s 90us/step - loss: 1.0756 - acc: 0.6817 - val_loss: 0.9532 - val_acc: 0.7050
Epoch 7/20
419967/419967 [==============================] - 38s 90us/step - loss: 1.0647 - acc: 0.6838 - val

In [ ]:
score = model.evaluate(x_test.reshape((x_test.shape[0],1,x_test.shape[1])), y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

In [ ]:
from sklearn.metrics import classification_report
y_predicted = model.predict_classes(x_test.reshape((x_test.shape[0],1,x_test.shape[1])), verbose=1)

In [ ]:
y_test_num = np.argmax(y_test, axis=1)

In [ ]:
print(classification_report(y_test_num, y_predicted))

In [ ]:
df_public = pd.read_csv('input/test.csv')
df_public.head(10)

In [ ]:
x_public = tokenize.texts_to_matrix(df_public["title"])

In [ ]:
preds = model.predict(x_public.reshape((x_public.shape[0],1,x_public.shape[1])), verbose=1)

In [ ]:
df_public['Category'] = [np.argmax(pred) for pred in preds]
df_submit = df_public[['itemid', 'Category']].copy()
df_submit.to_csv('submission.csv', index=False)

In [ ]:
model.save('ThurMar21.h5')